In [1]:
import pandas as pd
import numpy as np
import gudhi as gd
from gudhi.point_cloud.timedelay import TimeDelayEmbedding
from gudhi.hera import wasserstein_distance
from gudhi.representations import PersistenceImage
from gudhi.representations.metrics import WassersteinDistance
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn.manifold import MDS
import os
from tqdm.notebook import tqdm

%matplotlib widget

In [2]:
datapath =".\\data\\timeseries\\good_new"
filelist = os.listdir(datapath)
print(sorted(filelist))
good_time_series = [np.array(pd.read_csv(os.path.join(datapath,f), header=None)[0]) for f in sorted(filelist)]
datapath =".\\data\\timeseries\\bad_new"
filelist = os.listdir(datapath)
print(sorted(filelist))
bad_time_series = [np.array(pd.read_csv(os.path.join(datapath,f), header=None)[0]) for f in sorted(filelist)]

print(good_time_series[0])

['sig1_good.txt', 'sig2_good.txt', 'sig3_good.txt', 'sig4_good.txt', 'sig5_good.txt']
['sig1_bad.txt', 'sig2_bad.txt', 'sig3_bad.txt', 'sig4_bad.txt', 'sig5_bad.txt']
[-0.11156466  0.01043311  0.30969176 ...  0.44117724  0.98839737
 -2.1306849 ]


In [3]:
n_chunks = 200
chopped_good_ts = [[]]*len(good_time_series)
chopped_bad_ts = [[]]*len(bad_time_series)
chunks = np.array_split(range(0,len(good_time_series[0])), n_chunks)
for i in range(0,len(good_time_series)):
    chopped_good_ts[i] = np.array_split(good_time_series[i], n_chunks)
    chopped_bad_ts[i] = np.array_split(bad_time_series[i], n_chunks)

In [5]:
dim = 2
delay =7
skip = 1
good_point_clouds = [[]]*len(good_time_series)
bad_point_clouds = [[]]*len(bad_time_series)

tde = TimeDelayEmbedding(dim = dim, delay=delay, skip=skip)

for i in range(0,len(good_time_series)):
    good_point_clouds[i] = tde.transform(chopped_good_ts[i])
    bad_point_clouds[i] = tde.transform(chopped_bad_ts[i])

In [6]:
good_pds = [[]]*len(good_time_series)

for i in range(0,len(good_point_clouds)):
    for pc in good_point_clouds[i]:
        ac = gd.AlphaComplex(pc)
        st = ac.create_simplex_tree()
        st.compute_persistence()
        good_pds[i].append(st.persistence_intervals_in_dimension(1))

bad_pds = [[]]*len(bad_time_series)

for i in range(0,len(bad_point_clouds)):
    for pc in bad_point_clouds[i]:
        ac = gd.AlphaComplex(pc)
        st = ac.create_simplex_tree()
        st.compute_persistence()
        bad_pds[i].append(st.persistence_intervals_in_dimension(1))

In [8]:
outlier_chunks = []
dist_matrix = np.zeros((len(good_pds),n_chunks))
for j in range(0,len(good_pds)):
    print(j)
    for i in tqdm(range(0,n_chunks)):
        dist = gd.hera.wasserstein_distance(good_pds[j][i],bad_pds[j][i], internal_p = 2, order = 2)
        dist_matrix[j][i] = dist
        if dist > 0:
            print(i)
            print(dist)
            print(chunks[i])
            #outlier_chunks.append(i)

0


5
0.2630982874088608
[625 626 627 628 629 630 631 632 633 634 635 636 637 638 639 640 641 642
 643 644 645 646 647 648 649 650 651 652 653 654 655 656 657 658 659 660
 661 662 663 664 665 666 667 668 669 670 671 672 673 674 675 676 677 678
 679 680 681 682 683 684 685 686 687 688 689 690 691 692 693 694 695 696
 697 698 699 700 701 702 703 704 705 706 707 708 709 710 711 712 713 714
 715 716 717 718 719 720 721 722 723 724 725 726 727 728 729 730 731 732
 733 734 735 736 737 738 739 740 741 742 743 744 745 746 747 748 749]
6
5.8775099297758435e-08
[750 751 752 753 754 755 756 757 758 759 760 761 762 763 764 765 766 767
 768 769 770 771 772 773 774 775 776 777 778 779 780 781 782 783 784 785
 786 787 788 789 790 791 792 793 794 795 796 797 798 799 800 801 802 803
 804 805 806 807 808 809 810 811 812 813 814 815 816 817 818 819 820 821
 822 823 824 825 826 827 828 829 830 831 832 833 834 835 836 837 838 839
 840 841 842 843 844 845 846 847 848 849 850 851 852 853 854 855 856 857
 858 859

5
0.2630982874088608
[625 626 627 628 629 630 631 632 633 634 635 636 637 638 639 640 641 642
 643 644 645 646 647 648 649 650 651 652 653 654 655 656 657 658 659 660
 661 662 663 664 665 666 667 668 669 670 671 672 673 674 675 676 677 678
 679 680 681 682 683 684 685 686 687 688 689 690 691 692 693 694 695 696
 697 698 699 700 701 702 703 704 705 706 707 708 709 710 711 712 713 714
 715 716 717 718 719 720 721 722 723 724 725 726 727 728 729 730 731 732
 733 734 735 736 737 738 739 740 741 742 743 744 745 746 747 748 749]
6
5.8775099297758435e-08
[750 751 752 753 754 755 756 757 758 759 760 761 762 763 764 765 766 767
 768 769 770 771 772 773 774 775 776 777 778 779 780 781 782 783 784 785
 786 787 788 789 790 791 792 793 794 795 796 797 798 799 800 801 802 803
 804 805 806 807 808 809 810 811 812 813 814 815 816 817 818 819 820 821
 822 823 824 825 826 827 828 829 830 831 832 833 834 835 836 837 838 839
 840 841 842 843 844 845 846 847 848 849 850 851 852 853 854 855 856 857
 858 859

5
0.2630982874088608
[625 626 627 628 629 630 631 632 633 634 635 636 637 638 639 640 641 642
 643 644 645 646 647 648 649 650 651 652 653 654 655 656 657 658 659 660
 661 662 663 664 665 666 667 668 669 670 671 672 673 674 675 676 677 678
 679 680 681 682 683 684 685 686 687 688 689 690 691 692 693 694 695 696
 697 698 699 700 701 702 703 704 705 706 707 708 709 710 711 712 713 714
 715 716 717 718 719 720 721 722 723 724 725 726 727 728 729 730 731 732
 733 734 735 736 737 738 739 740 741 742 743 744 745 746 747 748 749]
6
5.8775099297758435e-08
[750 751 752 753 754 755 756 757 758 759 760 761 762 763 764 765 766 767
 768 769 770 771 772 773 774 775 776 777 778 779 780 781 782 783 784 785
 786 787 788 789 790 791 792 793 794 795 796 797 798 799 800 801 802 803
 804 805 806 807 808 809 810 811 812 813 814 815 816 817 818 819 820 821
 822 823 824 825 826 827 828 829 830 831 832 833 834 835 836 837 838 839
 840 841 842 843 844 845 846 847 848 849 850 851 852 853 854 855 856 857
 858 859

5
0.2630982874088608
[625 626 627 628 629 630 631 632 633 634 635 636 637 638 639 640 641 642
 643 644 645 646 647 648 649 650 651 652 653 654 655 656 657 658 659 660
 661 662 663 664 665 666 667 668 669 670 671 672 673 674 675 676 677 678
 679 680 681 682 683 684 685 686 687 688 689 690 691 692 693 694 695 696
 697 698 699 700 701 702 703 704 705 706 707 708 709 710 711 712 713 714
 715 716 717 718 719 720 721 722 723 724 725 726 727 728 729 730 731 732
 733 734 735 736 737 738 739 740 741 742 743 744 745 746 747 748 749]
6
5.8775099297758435e-08
[750 751 752 753 754 755 756 757 758 759 760 761 762 763 764 765 766 767
 768 769 770 771 772 773 774 775 776 777 778 779 780 781 782 783 784 785
 786 787 788 789 790 791 792 793 794 795 796 797 798 799 800 801 802 803
 804 805 806 807 808 809 810 811 812 813 814 815 816 817 818 819 820 821
 822 823 824 825 826 827 828 829 830 831 832 833 834 835 836 837 838 839
 840 841 842 843 844 845 846 847 848 849 850 851 852 853 854 855 856 857
 858 859

5
0.2630982874088608
[625 626 627 628 629 630 631 632 633 634 635 636 637 638 639 640 641 642
 643 644 645 646 647 648 649 650 651 652 653 654 655 656 657 658 659 660
 661 662 663 664 665 666 667 668 669 670 671 672 673 674 675 676 677 678
 679 680 681 682 683 684 685 686 687 688 689 690 691 692 693 694 695 696
 697 698 699 700 701 702 703 704 705 706 707 708 709 710 711 712 713 714
 715 716 717 718 719 720 721 722 723 724 725 726 727 728 729 730 731 732
 733 734 735 736 737 738 739 740 741 742 743 744 745 746 747 748 749]
6
5.8775099297758435e-08
[750 751 752 753 754 755 756 757 758 759 760 761 762 763 764 765 766 767
 768 769 770 771 772 773 774 775 776 777 778 779 780 781 782 783 784 785
 786 787 788 789 790 791 792 793 794 795 796 797 798 799 800 801 802 803
 804 805 806 807 808 809 810 811 812 813 814 815 816 817 818 819 820 821
 822 823 824 825 826 827 828 829 830 831 832 833 834 835 836 837 838 839
 840 841 842 843 844 845 846 847 848 849 850 851 852 853 854 855 856 857
 858 859

In [9]:
plt.matshow(dist_matrix)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
ts_dist_matrix = np.zeros((len(good_pds),n_chunks))
for j in range(0,len(good_time_series)):
    for i in range(0,n_chunks):
        diff = chopped_good_ts[j][i]-chopped_bad_ts[j][i]
        #print(j,i,diff)
        ts_dist_matrix[j][i] = np.abs(np.mean(diff))

In [15]:
plt.matshow(ts_dist_matrix)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …